In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
from torchvision import datasets, models, transforms
import torch.utils.data as data
import matplotlib.pyplot as plt

import pickle as pkl
import glob
import os
import copy
import time
from PIL import Image

from sklearn.metrics import confusion_matrix

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"

In [4]:
all_class = os.listdir("../Images/periodic/all")
print(len(all_class))

55


In [5]:
remove_classes = ["stone-wall4.o"]
test_classes   = ["B5CC4D9964F342BD.o"]
all_class = [ i for i in all_class if i not in remove_classes ]

train_target_classes = [ i for i in all_class if i not in test_classes ]
test_target_classes =  [ i for i in all_class if i in test_classes ]

print(len(train_target_classes))
print(len(test_target_classes))

53
1


In [7]:
train_packs = get_paths(train_target_classes)
test_packs  = get_paths(test_target_classes)
train_transforms = data_transformer_torch_train()
test_transforms  = data_transformer_torch_test()

In [8]:
datasets_train = Img_Dataset(file_list=train_packs[0],transform=train_transforms,labels=train_packs[1],class_labels=train_packs[2])
datasets_test  = Img_Dataset(file_list=test_packs[0] ,transform=test_transforms,labels=test_packs[1],class_labels=test_packs[2])

dataloader_train = torch.utils.data.DataLoader(datasets_train, batch_size=8, shuffle=True,num_workers=8)
dataloader_test  = torch.utils.data.DataLoader(datasets_test , batch_size=8, shuffle=False,num_workers=8)

dataloaders  = {"train":dataloader_train,"val":dataloader_test }
dataset_sizes ={"train":len(datasets_train),"val":len(datasets_test)}

In [9]:
model = models.vgg16(pretrained=True)
model.classifier = nn.Sequential(
        nn.Linear(25088,100),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(100,1) ,nn.Sigmoid())
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [10]:
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [3]:
best_acc = 0
num_epochs = 500

In [12]:
acc_dict  = {"train":[],"val":[]}
loss_dict = {"train":[],"val":[]}

for epoch in range(num_epochs):
    print("Epoch {}/{}".format(epoch+1,num_epochs))
    print("-"*10)
    
    for phase in ["train","val"]:
        print("---{}---".format(phase))
        sum_img = 0
        if phase == "train":
            scheduler.step()
            model.train()
        else:
            model.eval()
            
        running_loss = 0.0
        running_corrects = 0.
        
        for inputs, labels,_ in dataloaders[phase]:
            sum_img += inputs.size(0)
            print("{:6}/{:6}".format(sum_img,dataset_sizes[phase]),end="\r")
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase=="train"):
                preds = model(inputs)
                labels = labels.view_as(preds)
                loss = criterion(preds,labels)
                
                if phase == "train":
                    loss.backward()
                    optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects  += torch.sum( (preds>0.5) == labels ).item()
            
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc  = running_corrects / dataset_sizes[phase]   
        loss_dict[phase].append(epoch_loss)
        acc_dict[phase].append(epoch_acc)

        print('{} Loss: {:.4f} ,ACC:{:.4f}'.format(phase, epoch_loss,epoch_acc))

Epoch 1/500
----------
---train---


/home/yusuke/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 0.5589 ,ACC:0.6864
---val---
val Loss: 0.2929 ,ACC:1.0000
Epoch 2/500
----------
---train---
train Loss: 0.4580 ,ACC:0.7947
---val---
val Loss: 1.2064 ,ACC:0.5000
Epoch 3/500
----------
---train---
train Loss: 0.4002 ,ACC:0.8230
---val---
val Loss: 0.0741 ,ACC:1.0000
Epoch 4/500
----------
---train---
train Loss: 0.3167 ,ACC:0.8493
---val---
val Loss: 0.3345 ,ACC:0.8000
Epoch 5/500
----------
---train---
train Loss: 0.3199 ,ACC:0.8559
---val---
val Loss: 0.0410 ,ACC:1.0000
Epoch 6/500
----------
---train---
train Loss: 0.2362 ,ACC:0.8879
---val---
val Loss: 0.3229 ,ACC:0.8500
Epoch 7/500
----------
---train---


KeyboardInterrupt: 

In [ ]:
with open("models/log.pkl","wb") as f:
    pkl.dump([loss_dict,acc_dict],f)
torch.save(model.state_dict(),"models/final_model_wts.pt")